In [1]:
!pip install minerl

     |████████████████████████████████| 36.3MB 1.2MB/s 
     |████████████████████████████████| 788kB 42.5MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 5.5MB 45.7MB/s 
     |████████████████████████████████| 460kB 42.6MB/s 
     |████████████████████████████████| 92kB 9.6MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 13.0MB 193kB/s 
     |████████████████████████████████| 358kB 39.6MB/s 
     |████████████████████████████████| 92kB 9.9MB/s 
  Created wheel for minerl: filename=minerl-0.3.6-cp36-none-any.whl size=72040787 sha256=0afcadc0e3fd4cf92c0316a4105d16c7ba7012862efee9380f10b6b62d6e2d3b
  Stored in directory: /root/.cache/pip/wheels/8e/dc/c9/a7f3c17ccc05df396620a9e6921936d3bd35df1f25466c9fd7
  Created wheel for typing: filename=typing-3.7.4.3-cp36-none-any.whl size=26307 sha256=e275a054428c4a1453399caf6097268732b8a9e3a420ec6cec0df53fc2dbb3a0
  Stored in directory: /root/.cach

In [ ]:
from google.colab import drive
import minerl
import gym

import numpy as np

import collections

import pickle
import json

import os
os.environ['MINERL_DATA_ROOT']='./content/data'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -m minerl.data.download "MineRLNavigate-v0"

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'minerl.data.download' found in sys.modules after import of package 'minerl.data', but prior to execution of 'minerl.data.download'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2020-09-15 20:58:05 5d2ed16532b1 __main__[355] INFO Downloading experiment MineRLNavigate-v0 to ./content/data
2020-09-15 20:58:05 5d2ed16532b1 __main__[355] WARNING ./content/data/MineRLNavigate-v0 exists - skipping re-download!


In [ ]:
data = minerl.data.make('MineRLNavigate-v0')

In [ ]:
def create_default_actions():
  sample_actions = collections.OrderedDict()

  sample_actions['attack'] = 0
  sample_actions['back'] = 0
  sample_actions['camera'] = [0.,0.]
  sample_actions['forward'] = 0
  sample_actions['jump'] = 0
  sample_actions['left'] = 0
  sample_actions['place'] = 'none'
  sample_actions['right'] = 0
  sample_actions['sneak'] = 0
  sample_actions['sprint'] = 0

  return sample_actions

In [ ]:
def actions_to_dict(actions):
  actions_dict = collections.OrderedDict()

  actions_dict['attack'] = actions['attack']
  actions_dict['back'] = actions['back']
  actions_dict['camera'] = actions['camera']
  actions_dict['forward'] = actions['forward']
  actions_dict['jump'] = actions['jump']
  actions_dict['left'] = actions['left']
  actions_dict['place'] = actions['place']
  actions_dict['right'] = actions['right']
  actions_dict['sneak'] = actions['sneak']
  actions_dict['sprint'] = actions['sprint']

  return actions_dict

In [ ]:
def parse_demo(env, rep_buffer, data_path, nsteps=0):
  data = minerl.data.make(env)
  sequences = []
  i = 0

  for current_state, action, reward, next_state, done \
      in data.batch_iter(
          batch_size=1, num_epochs=1, seq_len=32):
          sequences.append({
              'curr_state':{'pov':current_state['pov'][0], 'compassAngle':current_state['compassAngle'][0], 'inventory':current_state['inventory']},
              'next_state':{'pov':next_state['pov'][0], 'compassAngle':next_state['compassAngle'][0], 'inventory':next_state['inventory']},
              'action':action,
              'reward':reward,
              'done':done,
              'sequence':i
            })
          i = i + 1
  return sequences

In [ ]:
sequences = parse_demo('MineRLNavigate-v0',[],0)

100%|██████████| 193/193 [00:00<00:00, 6384.53it/s]


In [ ]:
len(sequences)

7530

In [ ]:
def parse_sequences(sequences):
  sequences_copy = sequences
  reward_length = len(sequences[0]['reward'][0])
  done_length = len(sequences[0]['done'][0])
  action_length = len(sequences[0]['action']['camera'][0])
  sequences_length = len(sequences)
  sequences_2 = []
  

  for i in range(0, 100):
    actions = get_actions(sequences_copy[i]['action'], action_length)
    rewards = get_rewards(sequences_copy[i]['reward'], reward_length)
    dones = get_dones(sequences_copy[i]['done'], done_length)
    sequences_copy[i]['curr_state']['pov'] = sequences_copy[i]['curr_state']['pov']
    sequences_copy[i]['curr_state']['compassAngle'] = sequences_copy[i]['curr_state']['compassAngle']
    sequences_copy[i]['curr_state']['inventory']["dirt"] = sequences_copy[i]['curr_state']['inventory']["dirt"][0]
   
    sequences_copy[i]['next_state']['pov'] = sequences_copy[i]['next_state']['pov']
    sequences_copy[i]['next_state']['compassAngle'] = sequences_copy[i]['next_state']['compassAngle']
    sequences_copy[i]['next_state']['inventory']["dirt"] = sequences_copy[i]['next_state']['inventory']["dirt"][0]

    sequences_2.append({
        'sequence':i,
        'actions':actions,
        'rewards':rewards['rewards'],
        'dones':dones,
        'curr_states':sequences_copy[i]['curr_state'],
        'next_states':sequences_copy[i]['next_state'],
        'total_reward':rewards['total_reward']
    })
  
  return sequences_2

In [ ]:
def get_actions(actions, action_length):
  keys = list(actions.keys())
  all_actions = []

  for i in range(0, action_length):
    a = collections.OrderedDict()

    for key in keys:
      vals = 0
      if isinstance(actions[key][0][i], (np.ndarray)):
        vals = actions[key][0][i].tolist()
      else:
        if isinstance(actions[key][0][i],(np.int64)):
          vals = int(actions[key][0][i])
      
      a[key] = vals

    all_actions.append(a)

  return all_actions

In [ ]:
def get_rewards(rewards, rewards_length):
  all_rewards = {
      'rewards':[],
      'total_reward':0
  }

  for i in range(0, rewards_length):
    all_rewards['rewards'].append(float(rewards[0][i]))
    all_rewards['total_reward'] = all_rewards['total_reward'] + rewards[0][i]
  
  return all_rewards

In [ ]:
def get_dones(dones, dones_length):
  all_done = []

  for i in range(0, dones_length):
    all_done.append(str(dones[0][i]))

  return all_done

In [ ]:
type(sequences[0]['curr_state']['inventory']["dirt"][0].tolist())

list

In [ ]:
s = parse_sequences(sequences)

In [ ]:
import pickle

In [ ]:
filename = 'parsed_data.sav'
pickle.dump(s, open("/content/drive/My Drive/minerl-resources/datasets/"+filename, 'wb'))

In [ ]:
with open('dqfd_data.json', 'w') as f:
    json.dump(s, f)

print('done')

done
